# Inference
### Your interactive story

-> Try out station

---
### Installation

In [1]:
!python --version

Python 3.9.1


In [2]:
!python -m pip install --upgrade pip

DEPRECATION: tsfresh 0.18.0 has a non-standard dependency specifier matrixprofile>=1.1.10<2.0.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tsfresh or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!python -m pip install numpy
!python -m pip install torch
!python -m pip install transformers

DEPRECATION: tsfresh 0.18.0 has a non-standard dependency specifier matrixprofile>=1.1.10<2.0.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tsfresh or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: tsfresh 0.18.0 has a non-standard dependency specifier matrixprofile>=1.1.10<2.0.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tsfresh or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: tsfresh 0.18.0 has a non-standard dependency specifier matrixprofile>=1.1.10<2.0.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tsfresh or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


---
### Imports

In [4]:
import numpy
import torch
import transformers

---
### Initialization

In [5]:
MAX_LENGTH = 1024

tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2", max_length=MAX_LENGTH)
tokenizer.add_special_tokens({  "pad_token": "<pad>",
                                "bos_token": "<start>",
                                "eos_token": "<end>"})
tokenizer.add_tokens(["<bot>:"])

1

In [6]:
device = torch.device('cpu')

# loading
#config = transformers.GPT2Config.from_pretrained("gpt2")
#config.max_length = 883
model = transformers.GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.eval()
model = model.to(device)
model.load_state_dict(torch.load("./model/V5/model_state_V5_6.pt", map_location=device))

<All keys matched successfully>

In [7]:
out = tokenizer("test<bot><pad><pad>")
out

{'input_ids': [9288, 27, 13645, 29, 50257, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [8]:
def inference(prompt:str, model, tokenizer, device, padding, clear_output=True):
    model.eval()
    prompt = f"{prompt}<bot>"
    prompt = tokenizer(prompt, return_tensors="pt", padding=padding)
    X = prompt["input_ids"].to(device)
    a = prompt["attention_mask"].to(device)
    with torch.no_grad():
        output = model.generate(X, attention_mask=a, pad_token_id=tokenizer.pad_token_id, 
                                                        do_sample=True, max_length=MAX_LENGTH)

    if clear_output:
        output = tokenizer.decode(output[0], skip_special_tokens=True)
    else:
        output = tokenizer.decode(output[0], skip_special_tokens=False)

    if type(output) == list and len(output) == 1:
        output = output[0]
    return output

---
### Try out

In [9]:
output = inference(prompt="Hey, I'm feeling not so good.", model=model, tokenizer=tokenizer,
                                                        device=device, padding=True, clear_output=True)
output

"Hey, I'm feeling not so good.<bot>I'm sorry to hear that. What's going on?I've been feeling really overwhelmed lately with everything going on in my life.I understand. It's tough when everything seems to pile up at once. Can you tell me a little more about what's been happening?Well, I have been dealing with a lot of stress at work, and it's starting to take a toll on my mental health. I've also been having trouble sleeping and I feel like I'm constantly on edge.<bot>:It sounds like you're dealing with a lot right now. Have you tried any strategies to manage your stress levels?ongyang ideology"

In [11]:
loop = 0
while True:
    user_input = input()
    if user_input.lower() in ["q", "quit", "e", "exit", "", "x"]:
        print("See you later! I hope you had fun ^^")
        break
    elif user_input in ["restart", "new"]:
        loop = 0

    print("Bot:", inference(user_input, model=model, device=device, tokenizer=tokenizer, padding=True, clear_output=True))
    loop += 1

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
c:\Users\tobia\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1363: UserWarning: Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Bot: hey<bot>
See you later! I hope you had fun ^^


---